# Import Required Libraries

In [1]:
%pip install googletrans==4.0.0-rc1 --quiet
%pip install pandas --quiet
%pip install nltk --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import necessary libraries
from googletrans import Translator
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

# Load the CSV File

In [15]:
import pandas as pd

# Define the absolute path to the CSV file
csv_file_path = 'tweets.csv'

# Read the CSV file into a DataFrame
try:
    df = pd.read_csv(csv_file_path)
    # Display the DataFrame
    print(df.head(10))
except FileNotFoundError:
    print(f"File not found: {csv_file_path}")
except pd.errors.EmptyDataError:
    print("The file is empty.")
except Exception as e:
    print(f"An error occurred: {e}")
    
#print out the caption collumn
print(df['caption'])
    

               Post ID                                       URL to tweet  \
0  1828837775791341997  https://x.com/JimmyMatthewsJr/status/182883777...   
1  1828607916892422511  https://x.com/JimmyMatthewsJr/status/182860791...   
2  1826979750856049026  https://x.com/TheVladsven/status/1826979750856...   
3  1822071851960692972  https://x.com/TheVladsven/status/1822071851960...   
4  1822069995565883579  https://x.com/TheVladsven/status/1822069995565...   
5  1828212669670629624  https://x.com/Bloomer99999/status/182821266967...   
6  1828060378711945505  https://x.com/EmilyMach5/status/18280603787119...   
7  1813345368484073626  https://x.com/ScottWIsAnIdiot/status/181334536...   
8  1829862469894058286  https://x.com/EmilyMach5/status/18298624698940...   
9  1823835478480605640  https://x.com/acnewsitics/status/1823835478480...   

                    creation_date  likes  \
0  Wed Aug 28 16:51:18 +0000 2024     29   
1  Wed Aug 28 01:37:55 +0000 2024     21   
2  Fri Aug 23 13:48:

In [16]:

# Extract and remove mentions
df['mentions'] = df['caption'].apply(lambda x: ' '.join(re.findall(r'@\w+', x)))
df['caption_no_mentions'] = df['caption'].apply(lambda x: re.sub(r'@\w+', '', x))

# Extract hashtags
df['new_hashtags'] = df['caption_no_mentions'].apply(lambda x: ' '.join(re.findall(r'#\w+', x)))
df['hashtags'] = df['hashtags'].mask(df['hashtags'].isna() | (df['hashtags'] == ''), df['new_hashtags'])
df.drop(columns=['new_hashtags'], inplace=True)

# Extract and remove URLs
df['urls'] = df['caption_no_mentions'].apply(lambda x: ' '.join(re.findall(r'http\S+', x)))
df['urls'] = df['urls'].str.replace(r'https://t\.co/\w+', '', regex=True)
df['caption_clean'] = df['caption_no_mentions'].apply(lambda x: re.sub(r'http\S+', '', x))

#remove a ton of extra white space from caption_clean
df['caption_clean'] = df['caption_clean'].str.replace(r'\s+', ' ', regex=True)
#drop filtering collumns
df.drop(columns=['caption_no_mentions'], inplace=True)
df.rename(columns={'caption': 'original_caption'}, inplace=True)
print(df.head(10))


               Post ID                                       URL to tweet  \
0  1828837775791341997  https://x.com/JimmyMatthewsJr/status/182883777...   
1  1828607916892422511  https://x.com/JimmyMatthewsJr/status/182860791...   
2  1826979750856049026  https://x.com/TheVladsven/status/1826979750856...   
3  1822071851960692972  https://x.com/TheVladsven/status/1822071851960...   
4  1822069995565883579  https://x.com/TheVladsven/status/1822069995565...   
5  1828212669670629624  https://x.com/Bloomer99999/status/182821266967...   
6  1828060378711945505  https://x.com/EmilyMach5/status/18280603787119...   
7  1813345368484073626  https://x.com/ScottWIsAnIdiot/status/181334536...   
8  1829862469894058286  https://x.com/EmilyMach5/status/18298624698940...   
9  1823835478480605640  https://x.com/acnewsitics/status/1823835478480...   

                    creation_date  likes  \
0  Wed Aug 28 16:51:18 +0000 2024     29   
1  Wed Aug 28 01:37:55 +0000 2024     21   
2  Fri Aug 23 13:48:

# Sentiment Score

In [17]:

nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Perform sentiment analysis on the 'Text in English language' column and save the results to a new column
def analyze_sentiment(text):
    sentiment_scores = sia.polarity_scores(text)
    return sentiment_scores['pos'], sentiment_scores['neu'], sentiment_scores['neg'], sentiment_scores['compound']

# Apply the analyze_sentiment function to the 'Text in English language' column
df['Positive'], df['Neutral'], df['Negative'], df['Compound'] = zip(*df['caption_clean'].apply(analyze_sentiment))

# Display the first few rows of the DataFrame to verify the sentiment analysis
print(df.head())


               Post ID                                       URL to tweet  \
0  1828837775791341997  https://x.com/JimmyMatthewsJr/status/182883777...   
1  1828607916892422511  https://x.com/JimmyMatthewsJr/status/182860791...   
2  1826979750856049026  https://x.com/TheVladsven/status/1826979750856...   
3  1822071851960692972  https://x.com/TheVladsven/status/1822071851960...   
4  1822069995565883579  https://x.com/TheVladsven/status/1822069995565...   

                    creation_date  likes  \
0  Wed Aug 28 16:51:18 +0000 2024     29   
1  Wed Aug 28 01:37:55 +0000 2024     21   
2  Fri Aug 23 13:48:10 +0000 2024      1   
3  Sat Aug 10 00:45:56 +0000 2024     19   
4  Sat Aug 10 00:38:34 +0000 2024      8   

                                    original_caption  retweet_count  \
0  #TrumpIsUnfitForOffice #ArlingtonNationalCemet...             18   
1  @thedesertginger I would not be surprised if h...              5   
2  Respect marriage.  Donald Trump and Where's Me...       

[nltk_data] Downloading package vader_lexicon to C:\Users\Jonah
[nltk_data]     Dalton\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Import Spacy Packages

In [18]:
%pip install -U pip setuptools wheel
%pip install -U spacy
!python -m spacy download en_core_web_sm

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 16.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0
    Uninstalling pip-25.0:
      Successfully uninstalled pip-25.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---- ----------------------------------- 1.3/12.8 MB 7.4 MB/s eta 0:00:02
     ------- -------------------------------- 2.4/12.8 MB 6.4 MB/s eta 0:00:02
     ---------- ----------------------------- 3.4/12.8 MB 6.1 MB/s eta 0:00:02
     -------------- ------------------------- 4.7/12.8 MB 5.8 MB/s eta 0:00:02
     ------------------ --------------------- 5.8/12.8 MB 5.9 MB/s eta 0:00:02
     ---------------------- ----------------- 7.3/12.8 MB 6.0 MB/s eta 0:00:01
     --------------------------- ------

# Extract Entities

In [19]:
import spacy

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Function to perform NER and separate entities by type
def extract_entities_by_type(text):
    doc = nlp(text)
    persons = [entity.text for entity in doc.ents if entity.label_ == "PERSON"]
    orgs = [entity.text for entity in doc.ents if entity.label_ == "ORG"]
    gpes = [entity.text for entity in doc.ents if entity.label_ == "GPE"]
    rel = [entity.text for entity in doc.ents if entity.label_ == "NORP"]
    dat = [entity.text for entity in doc.ents if entity.label_ == "DATE"]
    return persons, orgs, gpes, rel, dat

# Apply the function to each tweet and store the results in separate columns
df[['Persons', 'Organizations', 'Locations', 'Nationalities/religiious groups', 'Date']] = df['caption_clean'].apply(
    lambda x: pd.Series(extract_entities_by_type(x))
)


# Topic Modeling

In [24]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import string
import re

# Sample dataset (replace this with your dataset)
import os

# Extract and remove mentions
df['topic_model_clean'] = df['original_caption'].apply(lambda x: re.sub(r'@\w+|http\S+', '', x) if pd.notnull(x) else '')

# List of words to remove (e.g., political keywords like 'trump')
stop_words_custom = ['trump', 'president', 'new', ]

# Preprocess text function (removes punctuation, stopwords, and other custom words)
def preprocess_text(text):
    if isinstance(text, str):  # Ensure the text is a string
        # Convert text to lowercase
        text = text.lower()
        
        # Remove punctuation
        text = ''.join([char for char in text if char not in string.punctuation])
        
        # Remove custom stop words
        text = ' '.join([word for word in text.split() if word not in stop_words_custom])
    
    return text

# Apply preprocessing only to the "cleaned_caption" column
df['topic_model_clean'] = df['topic_model_clean'].apply(preprocess_text)

# Vectorize the text using TF-IDF (you can adjust max_df, min_df to filter out too frequent/rare words)
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.80, min_df=3)
X = tfidf_vectorizer.fit_transform(df['topic_model_clean'])

# Train the LDA model (adjust num_topics, passes, iterations based on your dataset size)
num_topics = 4 # Optimal number of topics, you can experiment with this
lda_model = LatentDirichletAllocation(n_components=num_topics, max_iter=14, learning_method='online', random_state=42)

# Fit the LDA model to the TF-IDF matrix
lda_model.fit(X)

# Get top words for each topic
terms = tfidf_vectorizer.get_feature_names_out()
n_top_words = 10
topic_keywords = {}

topics = {
    0: "Postive Trump Sentiment",
    1: "Politcal Social Network",
    2: "Negative Candaite Sentiment",
    3: "Voter readiness and mobilization"
}

for topic_idx, topic in enumerate(lda_model.components_):
    top_words = [terms[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    topic_keywords[topic_idx] = top_words
    print(f"Topic #{topic_idx + 1} ({topics[topic_idx]}): {', '.join(top_words)}")

# Assign each document the most relevant topic
topic_assignments = lda_model.transform(X)
df['topic_id'] = topic_assignments.argmax(axis=1)

# Map numerical topics to labels
df['topic_label'] = df['topic_id'].map(topics)

# If the 'caption' column is blank, do not assign a topic
df.loc[df['original_caption'].isnull() | (df['original_caption'].str.strip() == ''), ['topic_id', 'topic_label']] = None

# Print the dataframe with topics assigned
print(df[['topic_model_clean', 'topic_label']].head(10))

# Remove topic_model_clean
df.drop('topic_model_clean', axis=1, inplace=True)


Topic #1 (Postive Trump Sentiment): maga, trump2024, maga2024, vote, peanut, protect, trumpvance2024, say, january, patriots
Topic #2 (Politcal Social Network): patriots, ifbap, trumpisunfitforoffice, let, day, trumpvance2024tosaveamerica, friends, free, ready, grow
Topic #3 (Negative Candaite Sentiment): fuck, time, remember, america, kamala2024, waiting, yes, breaking, happy, vote
Topic #4 (Voter readiness and mobilization): kamala, donald, like, election2024, real, thanks, think, elections2024, trumpvance2024, voted
                                   topic_model_clean  \
0    trumpisunfitforoffice arlingtonnationalcemetery   
1  i would not be surprised if he did this before...   
2  respect marriage donald and wheres melania alw...   
3  pray for he longs for north korea and his tub ...   
4  pray for he joined hamas 🇺🇸 trumpmeltdown trum...   
5                                  kamalawinswithjoy   
6  has used everyone around him for his personal ...   
7                          

# Save the New CSV File

In [25]:
# Select the required columns for the new CSV file

new_df = df[['caption_clean', 'image+caption_target', 'political_leaning', 'likes', 'retweet_count', 'comment_count', 'view_count', 'poster_follower_count', 'poster_frequency', 'total_posts', 'mentions', 'hashtags', 'urls', 
     'Positive', 'Neutral', 'Negative', 'Compound', 'Persons', 'Organizations', 'Locations', 'Nationalities/religiious groups', 'Date', 'topic_label', 'image_filename', 'comment_file', 'Post ID', 'URL to tweet', 'creation_date', 'poster_handle', 'poster_display_name',]]

# Save the new DataFrame to a CSV file
new_df.to_csv('Processed_Tweets.csv', index=False)